Notebook showing how to generate the Filter.xlsx file required when using a newer version of ecoinvent.

In [1]:
import pandas as pd
import numpy as np
import gzip
import pickle
import re

Fiets, download the python version of the newer ecoinvent to-be-used

In [2]:
with gzip.open('C://Users/Maxime/Desktop/Thesis/Bdd/Ecoinvent/python/ecoinvent3.7.1.cutoff/ecoinvent3.7.1.cut-offPandas_symmNorm.gz.pickle','rb') as f:
    ecoinvent = pd.read_pickle(f)

The piece of code below will determine which of the ecoinvent processes will be hybridized or not.

In [3]:
list_to_hyb = []
list_not_to_hyb = []
dummyprocesses = []
null_price = []
listmarket = []
listnottransacted = []
listguillotine = []


listproduct = ['sugar beet pulp', 'shavings', 'calcium chloride', 'pulverised lignite', 'heat pump, 30kW',
               'planning', 'saw dust', 'limestone, crushed', 'water, ultrapure', 'components for',
               'burned in refinery furnace', 'stone meal', 'grass', 'miscanthus', 'compressed',
               'evaporation', 'for planting', 'from zinc', 'zinc concentrate',
               'completely softened', 'tap water', 'heavy water', 'water, deionised', 'water, decarbonised',
               'ground granulated', 'deep well', 'collection service']
listactivity = ['basic slag', 'start-up', 'metals recovery', 'venting', 'nutrient supply',
                'recultivation', 'extrusion', 'excavation', 'transformation',
                'water production','thermoforming', 'blow moulding', 'sorting',
                'natural gas pressure reduction from high to low pressure']
listaddmarket = ['import', 'to generic market', 'to niche market', 'production mix', 'aluminium industry',
                 'cobalt industry','to market','electricity production, high voltage, for Swiss Federal Railways']

# calculating the sum of cost, we will compare it to the price and assert it's lower than the price
A_ff = ecoinvent['A'].copy()
A_ff = A_ff.fillna(0)
matrix = pd.concat([ecoinvent['PRO'].price.fillna(0)]*len(ecoinvent['PRO']),axis=1)
matrix.columns = ecoinvent['PRO'].index.tolist()
A_ff = A_ff.multiply(matrix)

for i in range(0, len(ecoinvent['PRO'])):
    try:
        # first, identify dummy processes, processes that do not have any inputs
        if ecoinvent['A'].loc[:,ecoinvent['PRO'].index[i]].sum() == 0:
            if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                dummyprocesses.append(ecoinvent['PRO'].index[i])
        # then those that do not have any price (we obviously cannot hybridize them)
        if ecoinvent['PRO'].price[i] == 0:
            if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                null_price.append(ecoinvent['PRO'].index[i])
        # market processes identification
        if (ecoinvent['PRO'].activityType[i] == 1
                or ecoinvent['PRO'].activityType[i] == 4
                or ecoinvent['PRO'].activityType[i] == 10):
            if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                listmarket.append(ecoinvent['PRO'].index[i])
        for j in range(0,len(listaddmarket)):
            if listaddmarket[j] in ecoinvent['PRO'].activityName[i]:
                if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                    list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                    listmarket.append(ecoinvent['PRO'].index[i])
                    break
        # non transacted goods/services identification
        for j in range(0, len(listproduct)):
            if listproduct[j] in ecoinvent['PRO'].productName[i]:
                if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                    list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                    listnottransacted.append(ecoinvent['PRO'].index[i])
                    break
        for k in range(0, len(listactivity)):
            if listactivity[k] in ecoinvent['PRO'].activityName[i]:
                if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                    list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                    listnottransacted.append(ecoinvent['PRO'].index[i])
                    break
        # identify intra-sectorial processes
        if (len(re.findall(r"[a-z]*[i][n][g]\b", ecoinvent['PRO'].activityName[i], flags=0)) != 0
            and 'production' not in ecoinvent['PRO'].activityName[i]
            and 'treatment' not in ecoinvent['PRO'].activityName[i]
            and 'waste' not in ecoinvent['PRO'].activityName[i]
            and 'construction' not in ecoinvent['PRO'].activityName[i]
            and 'installation' not in ecoinvent['PRO'].activityName[i]
            and 'plant' not in ecoinvent['PRO'].activityName[i]
            and 'facility' not in ecoinvent['PRO'].activityName[i]
            and ', building' not in ecoinvent['PRO'].activityName[i]
            and 'transport' not in ecoinvent['PRO'].activityName[i]
            and 'capture' not in ecoinvent['PRO'].activityName[i]
            or 'ventilation' in ecoinvent['PRO'].activityName[i]
            and 'production' not in ecoinvent['PRO'].activityName[i]
            and 'treatment' not in ecoinvent['PRO'].activityName[i]
            or 'energy' in ecoinvent['PRO'].activityName[i]
            and 'production' not in ecoinvent['PRO'].activityName[i]
            or ecoinvent['PRO'].unitName[i] == 'ha'
            or (ecoinvent['PRO'].unitName[i] == 'hour'
                and 'helicopter' not in ecoinvent['PRO'].productName[i])
            or len(re.findall(r'\b[s][e][e][d]', ecoinvent['PRO'].productName[i])) != 0
            or len(re.findall(r"^[0][1][6]", ecoinvent['PRO'].ISIC[i])) != 0
            or ('operation' in ecoinvent['PRO'].activityName[i]
                and ('mine' not in ecoinvent['PRO'].activityName[i]
                     and 'petroleum' not in ecoinvent['PRO'].activityName[i]
                     and 'quarry' not in ecoinvent['PRO'].activityName[i]
                     and 'pit' not in ecoinvent['PRO'].activityName[i]))
            or 'maintenance' in ecoinvent['PRO'].productName[i]
                and ('bus' not in ecoinvent['PRO'].productName[i]
                     and 'vehicle' not in ecoinvent['PRO'].productName[i]
                     and 'lorry' not in ecoinvent['PRO'].productName[i]
                     and 'passenger car' not in ecoinvent['PRO'].productName[i]
                     and 'electric bicycle' not in ecoinvent['PRO'].productName[i]
                     and 'scooter' not in ecoinvent['PRO'].productName[i]
                     and 'trawler' not in ecoinvent['PRO'].productName[i])):
            if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                listnottransacted.append(ecoinvent['PRO'].index[i])
        # guillotines, quality of process is questionable
        if (ecoinvent['PRO'].productName[i] != 'electricity, high voltage'
            and ecoinvent['PRO'].productName[i] != 'electricity, medium voltage'
            and ecoinvent['PRO'].productName[i] != 'electricity, low voltage'):
            if (ecoinvent['PRO'].price[i] > 100000
                    or A_ff.iloc[:, i].sum() > ecoinvent['PRO'].price[i]
                    or A_ff.iloc[:, i].sum() < 0.01 * ecoinvent['PRO'].price[i]):
                if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                    list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                    listguillotine.append(ecoinvent['PRO'].index[i])

    except TypeError:
        if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
            print('by default the process: '+ecoinvent['PRO'].index[i]+
                  ' was put in listnottransacted, because no sector has been entered.')
            list_not_to_hyb.append(ecoinvent['PRO'].index[i])
            listnottransacted.append(ecoinvent['PRO'].index[i])

for i in range(0, len(ecoinvent['PRO'])):
    if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
        list_to_hyb.append(ecoinvent['PRO'].index[i])

Here we identify potential new geographies of new ecoinvent version. We only need to look at processes within list_to_hyb, others don't matter. Also, RoW has a particular status so it does not count

In [4]:
dict1 = open('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco3.7.1_exio3/geography_replacements.txt')
dict1 = eval(dict1.read())
dict2 = open('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco3.7.1_exio3/countries_per_regions.txt')
dict2 = eval(dict2.read())
list3 = open('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco3.7.1_exio3/countries.txt')
list3 = eval(list3.read())

covered_geos = list(dict1.keys())+list(dict2.keys())+list3

set(ecoinvent['PRO'].loc[[i for i in ecoinvent['PRO'].index if (
    ecoinvent['PRO'].loc[i,'geography'] not in covered_geos and i in list_to_hyb)]].geography)-set(['RoW'])

set()

If the set is not empty, it means there are some new geographies used by the newer version of ecoinvent that must be linked to the geographies from exiobase. We must do this "by hand" and put them in the geography_replacements.txt file.

We do not hybridize processes that are matched to rest-of-the-world regions from exiobase

In [5]:
ecoinvent['PRO']['io_geo'] = [dict1[i] if i in dict1 else i for i in ecoinvent['PRO'].geography]
uncoveredgeoDF = ecoinvent['PRO'].loc[[i for i in ecoinvent['PRO'].index if (ecoinvent['PRO'].loc[i,'io_geo'] in ['WA','WE','WF','WL','WM']
                    and i in list_to_hyb)],['price', 'geography','productName', 'activityName']]
# they are not hybridized anymore so remove them from the list
for process in uncoveredgeoDF.index:
    list_to_hyb.remove(process)

Create the Filter.xlsx file

In [6]:
hybridizedDF = ecoinvent['PRO'].loc[list_to_hyb,['price', 'geography','productName', 'activityName']]
marketDF = ecoinvent['PRO'].loc[listmarket,['price', 'geography','productName', 'activityName']]
nottransactedDF = ecoinvent['PRO'].loc[listnottransacted,['price', 'geography','productName', 'activityName']]
guillotineDF = ecoinvent['PRO'].loc[listguillotine,['price', 'geography','productName', 'activityName']]
dummyDF = ecoinvent['PRO'].loc[dummyprocesses,['price', 'geography','productName', 'activityName']]
nullpriceDF = ecoinvent['PRO'].loc[null_price,['price', 'geography','productName', 'activityName']]

writer = pd.ExcelWriter('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco3.7.1_exio3/Filter.xlsx', engine='xlsxwriter')
hybridizedDF.to_excel(writer, sheet_name='Hybridized')
marketDF.to_excel(writer, sheet_name='Market')
nottransactedDF.to_excel(writer, sheet_name='Not commercialized')
guillotineDF.to_excel(writer, sheet_name='Poor quality')
dummyDF.to_excel(writer, sheet_name='Empty processes')
nullpriceDF.to_excel(writer, sheet_name='No price')
uncoveredgeoDF.to_excel(writer, sheet_name='Uncovered geography')
writer.save()